In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/a0m02fp/pt_list_dump_image_urls.tsv', sep='\\t', engine='python')

In [ ]:
fn = lambda obj: obj.loc[np.random.choice(obj.index, min(len(obj.index), 8000), False),:]
df_sample = df.groupby('product_type', as_index=False).apply(fn)

In [ ]:
df_sample.to_csv("non_placeholder_image_urls.csv", sep=",")

In [ ]:
import json
prod_urls = []
for url in urls:
    if isinstance(url, basestring):
        x = json.loads(url)
        if len(x) > 1:
            prod_urls.append(x[1])
        else:
            prod_urls.append(x[0])

In [ ]:
import os
with open("non_placeholder_image_urls.txt", 'w') as f:
    for url in prod_urls:
        f.write(url + os.linesep)

In [ ]:
from keras.utils import Sequence

class Image_Array_Generator(Sequence):
    def __init__(self, data_pairs, batch_size):
        self.data_pairs = data_pairs
        self.batch_size = batch_size

    def __len__(self):
        return np.ceil(len(self.data_pairs) / float(self.batch_size))

    def __getitem__(self, idx):
        batch_data_pairs = self.data_pairs[idx * self.batch_size:(idx + 1) * self.batch_size]
        urls1, urls2, labels = zip(*batch_data_pairs)
        
        image_data1 = np.array([img_to_array(load_img(url).resize((128, 128))) for url in urls1])/255.0
        image_data2 = np.array([img_to_array(load_img(url).resize((128, 128))) for url in urls2])/255.0

        return [image_data1, image_data2], np.array(labels)

In [ ]:
training_batch_generator = Image_Array_Generator(train_data_pairs, 256)
testing_batch_generator = Image_Array_Generator(test_data_pairs, 256)

In [40]:
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Model, Input
from keras.layers import Dropout, Flatten, Dense, Activation, concatenate, Lambda, merge
from keras.layers.convolutional import Conv2D, MaxPooling2D
import keras.backend as K
from keras.regularizers import l2
from keras.optimizers import Adam

input1, input2 = Input(shape=(64,64,3)), Input(shape=(64,64,3))

conv = Conv2D(filters=32, kernel_size=(11, 11), padding='same', activation='relu', kernel_regularizer=l2(2e-4))
conv_a, conv_b = conv(input1), conv(input2)

max_pool = MaxPooling2D(pool_size=(2, 2))
max_pool_a, max_pool_b = max_pool(conv_a), max_pool(conv_b)

conv = Conv2D(filters=64, kernel_size=(7, 7), padding='same', activation='relu', kernel_regularizer=l2(2e-4))
conv_a, conv_b = conv(max_pool_a), conv(max_pool_b)

max_pool = MaxPooling2D(pool_size=(2, 2))
max_pool_a, max_pool_b = max_pool(conv_a), max_pool(conv_b)


flatten = Flatten()
flatten_a, flatten_b = flatten(max_pool_a), flatten(max_pool_b)

dense = Dense(512, activation="sigmoid", kernel_regularizer=l2(1e-3))
dense_a, dense_b = dense(flatten_a), dense(flatten_b)

merged = Lambda(lambda x: K.abs(x[0]-x[1]))([dense_a, dense_b])

out = Dense(1, activation="sigmoid")(merged)

model = Model(inputs=[input1, input2], outputs=out)

optimizer = Adam(lr=0.00005, clipnorm=5.)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit([image_data1, image_data2], labels, batch_size=256, epochs=5, validation_split=0.1, verbose=1)

model.fit_generator(generator=training_batch_generator,
                  steps_per_epoch=(len(train_data_pairs) // 256),
                  epochs=5,
                  verbose=1,
                  validation_data=testing_batch_generator,
                  validation_steps=(len(test_data_pairs) // 256),
                  use_multiprocessing=True,
                  workers=5,
                  max_queue_size=32)

Train on 6512 samples, validate on 724 samples
Epoch 1/5
6512/6512 [==============================] - 89s 14ms/step - loss: 1.5266 - acc: 0.7299 - val_loss: 1.3599 - val_acc: 0.7942
Epoch 2/5
6512/6512 [==============================] - 86s 13ms/step - loss: 1.2675 - acc: 0.8394 - val_loss: 1.1520 - val_acc: 0.8412
Epoch 3/5
6512/6512 [==============================] - 87s 13ms/step - loss: 1.1078 - acc: 0.8713 - val_loss: 1.0922 - val_acc: 0.8384
Epoch 4/5
6512/6512 [==============================] - 88s 14ms/step - loss: 0.9870 - acc: 0.8934 - val_loss: 0.9231 - val_acc: 0.9006
Epoch 5/5
6512/6512 [==============================] - 87s 13ms/step - loss: 0.8950 - acc: 0.9049 - val_loss: 0.9004 - val_acc: 0.8771


In [ ]:
from PIL import Image
import glob, hashlib, os

d_placeholder_images = glob.glob("downloaded_placeholder_images/*.*")

for img_path in d_placeholder_images:
    im = Image.open(img_path)
    rgb_im = im.convert('RGB')
    md5_hash = hashlib.md5(img_path.encode('utf-8')).hexdigest()
    rgb_im.save(os.path.join("downloaded_placeholder_images" , md5_hash + '.jpg'))

In [1]:
import importlib
import unimodel_classifier
importlib.reload(unimodel_classifier)

from unimodel_classifier import UniModel
import data_generator as dg
importlib.reload(dg)

import numpy as np

image_data, labels = dg.get_image_data_unimodel('test')

image_shape = (image_data.shape[1], image_data.shape[2], image_data.shape[3])

unimodel = UniModel("data/unimodel.h5")
unimodel.init_model(image_shape)

/libs/base/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
unimodel.load()
unimodel.score(image_data, labels)

ResourceExhaustedError: OOM when allocating tensor with shape[1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dense_2/bias/Assign (defined at /home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:402)  = Assign[T=DT_FLOAT, _grappler_relax_allocator_constraints=true, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_2/bias, dense_2/Const)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_2/bias/Assign', defined at:
  File "/usr/lib64/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib64/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jupyter/libraries/nb_env/lib64/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.l

In [116]:
import siamese_predictor
reload(siamese_predictor)

from siamese_predictor import SiameseModel
import data_generator as dg
reload(dg)

import numpy as np

image_data, labels = dg.get_image_data_siamese('test')

image_shape = (image_data[0].shape[1], image_data[0].shape[2], image_data[0].shape[3])

siamese_model = SiameseModel("data/siamese_model.h5")
siamese_model.init_model(image_shape)

In [118]:
siamese_model.load()
siamese_model.score(image_data, labels)

             precision    recall  f1-score   support

          0       0.97      0.79      0.87      4576
          1       0.82      0.98      0.89      4576

avg / total       0.89      0.88      0.88      9152



In [129]:
import hashing_predictor as hp
reload(hp)
from hashing_predictor import Hashing
import glob, os

hashing = Hashing(threshold=15)
train_image_files = glob.glob(os.path.join("tagged_placeholder_images", "*.*"))
hashing.fit(train_image_files)

In [133]:
import random

hashing.threshold = 15
pos_image_files = glob.glob(os.path.join("similar_placeholder_images", "*.*"))
neg_image_files = glob.glob(os.path.join("product_images", "*.*"))

print hashing.score(pos_image_files, neg_image_files)

             precision    recall  f1-score   support

          0       0.96      0.97      0.97      3863
          1       0.97      0.96      0.96      3263

avg / total       0.97      0.97      0.97      7126

None


In [4]:
import numpy as np
np.repeat([[1,2,3,4]], 4, axis=0)

array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4]])

In [1]:
1e-2

0.01

In [1]:
100000/256.0

390.625

In [4]:
import math
int(math.ceil(float(100000)/256))

391